In [1]:
import pandas as pd; import numpy as np; import janitor

In [2]:
df = pd.DataFrame({
    "a": ["bla1", "nums", 6, 9],
    "b": ["bla2", "chars", "x", "y"],
})
df

,a,b
0,bla1,bla2
1,nums,chars
2,6,x
3,9,y


In [7]:
df.row_to_names([0,2], remove_rows=True,remove_rows_above=True)


/home/sam/pyjanitor/janitor/functions/row_to_names.py:87: UserWarning: The function row_to_names will, in the official 1.0 release, change its behaviour to reset the dataframe's index by default. You can prepare for this change right now by explicitly setting `reset_index=True` when calling on `row_to_names`.
  warnings.warn(


ValueError: The remove_rows_above argument is applicable only if the row_numbers argument is an integer, or the integers in a list are consecutive increasing, with a difference of 1.

In [9]:
data = {
        "a": [1, 2, 3] * 3,
        "Bell__Chart": [1.234_523_45, 2.456_234, 3.234_612_5] * 3,
        "decorated-elephant": [1, 2, 3] * 3,
        "animals@#$%^": ["rabbit", "leopard", "lion"] * 3,
        "cities": ["Cambridge", "Shanghai", "Basel"] * 3,
    }
df = pd.DataFrame(data)

df


,a,Bell__Chart,decorated-elephant,animals@#$%^,cities
0,1,1.234523,1,rabbit,Cambridge
1,2,2.456234,2,leopard,Shanghai
2,3,3.234612,3,lion,Basel
3,1,1.234523,1,rabbit,Cambridge
4,2,2.456234,2,leopard,Shanghai
5,3,3.234612,3,lion,Basel
6,1,1.234523,1,rabbit,Cambridge
7,2,2.456234,2,leopard,Shanghai
8,3,3.234612,3,lion,Basel


In [10]:
df.row_to_names([1,2])

/home/sam/pyjanitor/janitor/functions/row_to_names.py:87: UserWarning: The function row_to_names will, in the official 1.0 release, change its behaviour to reset the dataframe's index by default. You can prepare for this change right now by explicitly setting `reset_index=True` when calling on `row_to_names`.
  


,2,2.456234,2,leopard,Shanghai
,3,3.234612,3,lion,Basel
0,1,1.234523,1,rabbit,Cambridge
1,2,2.456234,2,leopard,Shanghai
2,3,3.234612,3,lion,Basel
3,1,1.234523,1,rabbit,Cambridge
4,2,2.456234,2,leopard,Shanghai
5,3,3.234612,3,lion,Basel
6,1,1.234523,1,rabbit,Cambridge
7,2,2.456234,2,leopard,Shanghai
8,3,3.234612,3,lion,Basel


In [13]:
df = pd.DataFrame({
             "a": ["nums", 6, 9],
             "b": ["chars", "x", "y"],
         })

df.row_to_names([0,1],remove_rows=True,reset_index=True)

/home/sam/pyjanitor/janitor/functions/row_to_names.py:87: UserWarning: The function row_to_names will, in the official 1.0 release, change its behaviour to reset the dataframe's index by default. You can prepare for this change right now by explicitly setting `reset_index=True` when calling on `row_to_names`.
  


,nums,chars
,6,x
0,9,y


In [4]:
df

,a,b
0,bla1,bla2
1,nums,chars
2,6,x
3,9,y


In [5]:
df.index.symmetric_difference([1,2])

Index([0, 3], dtype='int64')